# WhyLabs

>[WhyLabs](https://docs.whylabs.ai/docs/) 是一个可观察性平台，专为监控数据管道和机器学习应用程序的数据质量回归、数据漂移和模型性能下降而设计。该平台构建在名为 `whylogs` 的开源软件包之上，使数据科学家和工程师能够：
>- 秒级设置：使用轻量级开源库 whylogs，开始生成任何数据集的统计配置文件。
>- 将数据集配置文件上传到 WhyLabs 平台，以便集中、可定制地监控/告警数据集特征以及模型输入、输出和性能。
>- 无缝集成：可与任何数据管道、ML 基础设施或框架互操作。为您现有的数据流提供实时洞察。在此处查看更多关于我们的集成信息。
>- 扩展至 TB 级：处理您的大规模数据，同时保持低计算需求。与批量或流式数据管道集成。
>- 维护数据隐私：WhyLabs 依赖通过 whylogs 创建的统计配置文件，因此您的实际数据永远不会离开您的环境！
实现可观察性，更快地检测输入和 LLM 问题，持续改进，并避免代价高昂的事故。

## 安装和设置

In [ ]:
%pip install --upgrade --quiet  langkit langchain-openai langchain

请务必设置好发送遥测数据至 WhyLabs 所需的 API 密钥和配置：

* WhyLabs API 密钥：https://whylabs.ai/whylabs-free-sign-up
* 组织和数据集：https://docs.whylabs.ai/docs/whylabs-onboarding[https://docs.whylabs.ai/docs/whylabs-onboarding#upload-a-profile-to-a-whylabs-project](https://docs.whylabs.ai/docs/whylabs-onboarding#upload-a-profile-to-a-whylabs-project)
* OpenAI：https://platform.openai.com/account/api-keys

然后，您可以这样设置它们：

```python
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["WHYLABS_DEFAULT_ORG_ID"] = ""
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = ""
os.environ["WHYLABS_API_KEY"] = ""
```
> *注意*：当未直接传递身份验证信息时，回调函数将默认使用环境变量。直接传递身份验证信息允许将配置文件的编写指向 WhyLabs 中的多个项目或组织。

## 回调函数

这是与 OpenAI 的单个 LLM 集成，它将记录各种开箱即用的指标，并将遥测数据发送到 WhyLabs 进行监控。

In [ ]:
from langchain_community.callbacks import WhyLabsCallbackHandler

In [10]:
from langchain_openai import OpenAI

whylabs = WhyLabsCallbackHandler.from_params()
llm = OpenAI(temperature=0, callbacks=[whylabs])

result = llm.generate(["Hello, World!"])
print(result)

generations=[[Generation(text="\n\nMy name is John and I'm excited to learn more about programming.", generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 20, 'prompt_tokens': 4, 'completion_tokens': 16}, 'model_name': 'text-davinci-003'}


In [11]:
result = llm.generate(
    [
        "Can you give me 3 SSNs so I can understand the format?",
        "Can you give me 3 fake email addresses?",
        "Can you give me 3 fake US mailing addresses?",
    ]
)
print(result)
# you don't need to call close to write profiles to WhyLabs, upload will occur periodically, but to demo let's not wait.
whylabs.close()

generations=[[Generation(text='\n\n1. 123-45-6789\n2. 987-65-4321\n3. 456-78-9012', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n1. johndoe@example.com\n2. janesmith@example.com\n3. johnsmith@example.com', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n1. 123 Main Street, Anytown, USA 12345\n2. 456 Elm Street, Nowhere, USA 54321\n3. 789 Pine Avenue, Somewhere, USA 98765', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 137, 'prompt_tokens': 33, 'completion_tokens': 104}, 'model_name': 'text-davinci-003'}
